# UAS

## Crawling Portal Berita Antaranews

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import random

- request: Library ini memungkinkan Anda untuk mengirim HTTP requests (seperti GET, POST, PUT, DELETE) dan menerima respons dari server. Dengan menggunakan Requests, Anda dapat mengakses dan mengambil data dari URL, mengirim data form, dan berinteraksi dengan API.

- beautifulSoup: Beautiful Soup adalah library Python yang digunakan untuk mengambil data dari HTML dan XML. Ini menyediakan cara yang mudah untuk menjelajahi, mencari, dan memanipulasi struktur data dokumen web. Beautiful Soup bekerja dengan parser HTML, membantu Anda mengekstrak informasi dari halaman web dengan lebih mudah.

- csv: Modul CSV di Python menyediakan fungsi untuk membaca dan menulis file CSV. Ini memungkinkan Anda untuk dengan mudah bekerja dengan data dalam format CSV, yang umumnya digunakan untuk menyimpan data tabel, terutama dalam konteks analisis data dan pertukaran data antar program.

- random: Library ini menyediakan fungsi-fungsi untuk menghasilkan angka acak. Dalam konteks web scraping atau pengujian, Anda mungkin menggunakan library ini untuk memilih elemen acak dari daftar atau untuk menyimulasikan perilaku yang tidak terduga.



In [ ]:
# masukkan url
response = requests.get("https://www.antaranews.com/")
# Isi teks dari respons HTTP yang diterima dari server web setelah melakukan permintaan GET.
soup = BeautifulSoup(response.text, 'html.parser')
# menemukan semua list yang berisi link kategori
first_page = soup.findAll('li',"dropdown mega-full menu-color1")

# menyimpan kategori
save_categori = []
for links in first_page:
  categori = links.find('a').get('href')
  save_categori.append(categori)
# save_categori

# categori yang akan disearch terdapat pada indeks 1 (politik)
categori_search = [save_categori[1]]
categori_search

In [ ]:
# Inisialisasi list untuk menyimpan data berita
datas = []

# Iterasi melalui halaman berita
for ipages in range(1, 3):

    # Iterasi melalui setiap kategori berita
    for beritas in categori_search:
        # Permintaan untuk halaman berita
        response_berita = requests.get(beritas + "/" + str(ipages))
        namecategori = beritas.split("/")

        # Parsing halaman berita dengan BeautifulSoup
        soup_berita = BeautifulSoup(response_berita.text, 'html.parser')
        pages_berita = soup_berita.findAll('article', {'class': 'simple-post simple-big clearfix'})

        # Iterasi melalui setiap artikel dalam halaman berita
        for items in pages_berita:
            # Mendapatkan link artikel
            get_link_in = items.find("a").get("href")

            # Request untuk halaman artikel
            response_artikel = requests.get(get_link_in)
            soup_artikel = BeautifulSoup(response_artikel.text, 'html.parser')

            # Ekstraksi informasi dari halaman artikel
            judul = soup_artikel.find("h1", "post-title").text if soup_artikel.findAll("h1", "post-title") else ""
            label = namecategori[-1]
            date = soup_artikel.find("span", "article-date").text if soup_artikel.find("span", "article-date") else "Data tanggal tidak ditemukan"

            trash1 = ""
            cek_baca_juga = soup_artikel.findAll("span", "baca-juga")
            if cek_baca_juga:
                for bacas in cek_baca_juga:
                    text_trash = bacas.text
                    trash1 += text_trash + ' '

            artikels = soup_artikel.find_all('div', {'class': 'post-content clearfix'})
            artikel_content = artikels[0].text if artikels else ""
            artikel = artikel_content.replace("\n", " ").replace("\t", " ").replace("\r", " ").replace(trash1, "").replace("\xa0", "")

            author = soup_artikel.find("p", "text-muted small mt10").text.replace("\t\t", "") if soup_artikel.findAll("p", "text-muted small mt10") else ""

            # Menambahkan data artikel ke dalam list
            datas.append({'Tanggal': date, 'Penulis': author, 'Judul': judul, 'Artikel': artikel, 'Label': label})

In [ ]:
# Iterasi melalui halaman berita
for ipages in range(1, 3):
    # Mengecek apakah iterasi saat ini kurang dari 2
    if ipages < 2:
        # Menampilkan jumlah data yang telah berhasil di-crawling
        print(f'Data berhasil dicrawling sebanyak : {len(datas)}')

In [ ]:
# Menyimpan data dalam bentuk CSV
csv_filename = 'berita_politik_antaranews.csv'
# Membuka file CSV untuk ditulis ('w' mode) dengan encoding utf-8
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    # Menentukan kolom (fieldnames) untuk file CSV
    fieldnames = ['Tanggal', 'Penulis', 'Judul', 'Artikel', 'Label']
    # Membuat objek DictWriter untuk menulis data ke file CSV
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    # Menulis header (baris pertama) ke file CSV
    csv_writer.writeheader()
    # Iterasi melalui data yang akan disimpan (diasumsikan datas sudah didefinisikan sebelumnya)
    for entry in datas:
        # Menulis setiap baris data ke file CSV
        csv_writer.writerow(entry)
# Menampilkan pesan bahwa data telah disimpan dalam file CSV
print(f"Data telah disimpan dalam file {csv_filename}")

## Import library/package yang dibutuhkan (Memproses Data)

In [ ]:
import pandas as pd
import re

- Pandas adalah library open-source yang ditulis dalam bahasa pemrograman Python untuk analisis data dan manipulasi data. Dikembangkan oleh Wes McKinney, Pandas menyediakan struktur data yang efisien dan mudah digunakan untuk memanipulasi dan menganalisis data numerik dan tabular. Pandas sangat populer di kalangan data scientist, analis data, dan pengembang perangkat lunak karena kemudahan penggunaannya dan dukungan yang luas dalam ekosistem Python

- re adalah library Python yang menyediakan dukungan untuk ekspresi reguler (regular expressions). Ekspresi reguler adalah urutan karakter yang membentuk pola pencarian. Modul re memungkinkan pencocokkan pola dengan string dan melakukan berbagai operasi seperti pencarian, pencocokan, dan manipulasi string berdasarkan pola tertentu.